In [1]:
import torch
import pickle
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import os
import config
import utils
import main_multi as mt
from tqdm import tqdm


# seed 고정
random_seed = 42

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [2]:
SOURCE_DATASET = 'BeijingPM'

data_root_dir = f'./data/{SOURCE_DATASET}/'
file = [file for file in os.listdir(data_root_dir) if file.endswith('.csv')]
data = pd.read_csv(os.path.join(data_root_dir, file[0]))
data.dropna(inplace=True)
data = data.reset_index(drop=True)

data = data[[ 'DEWP', 'TEMP', 'PRES', 'Iws', 'pm2.5', 'cbwd']] # Multi-task learning

data[['cbwd']].value_counts() # 4 class classification

def sequence_preprocessing(data_x, data_y_1, data_y_2, timestep, shift_size):
    X = []
    targets_1 = []
    targets_2 = []

    # Slicing
    for start_idx in  tqdm(range(0, data_x.shape[0] - timestep + 1, shift_size)):
        X.append(data_x[start_idx:start_idx + timestep])

        ### Method1. Last (Window의 마지막 값을 Label로 활용)

        targets_1.append(data_y_1.values[start_idx + timestep - 1])
        targets_2.append(data_y_2.values[start_idx + timestep - 1])

    # Make to array 
    X = np.array(X)
    targets_1 = np.array(targets_1)
    targets_2 = np.array(targets_2)
    

    # (Instace, Features, Timestep)
    X = X.transpose(0, 2, 1)

    return X, targets_1, targets_2

data_target = data.copy()

data_x = data_target[['DEWP', 'TEMP', 'PRES', 'Iws']]
data_y_1 = data_target[['cbwd']]
data_y_2 = data_target[['pm2.5']]

seq_len = 24
x, y_1, y_2 = sequence_preprocessing(data_x, data_y_1, data_y_2, timestep=seq_len, shift_size=1)

label_encoder = LabelEncoder()
y_1 = label_encoder.fit_transform(y_1)

data_type = 'data_multi'

if data_type == 'data_single_1':
    x = x.copy()
    y = y_1.copy()
    
elif data_type == 'data_single_2':
    x = x.copy()
    y = y_2.copy()

elif data_type == 'data_multi':
    x = x.copy()
    y = np.concatenate([y_1.reshape(-1,1), y_2], axis=1)
    
split_ratio = 0.2
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=split_ratio, shuffle=False, random_state=502)
# TODO: Add scaler

train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=split_ratio, shuffle=True, random_state=502)

input_size = train_x.shape[1]

if data_type == 'data_single_1':
    num_classes = 1

elif data_type == 'data_single_2':
    num_classes = 4

    
else:
    num_classes_1 = 4
    num_classes_2 = 1
    

100%|██████████| 41734/41734 [00:01<00:00, 25745.82it/s]


# Pre-training

In [6]:
# Classification pre-training

model_name = 'LSTM_FCNs_multi'
model_params = config.model_config[model_name]

model_params['parameter']['input_size'] = input_size
model_params['parameter']['alpha'] = 1 # cls = alpha, reg = 1-alpha
model_params['best_model_path'] = f'./ckpt/{SOURCE_DATASET}/lstm_fcn_multi_{seq_len}_pre_cls.pt'

data_source = mt.Multilearning(model_params,'self')
data_source.build_model()

best_model = data_source.train_model(train_x, train_y, valid_x, valid_y)  # 모델 학습

os.makedirs(f'./ckpt/{SOURCE_DATASET}/', exist_ok=True)
data_source.save_model(best_model, best_model_path=model_params["best_model_path"])  # 모델 저장



Epoch 1/500
train Loss: 1.1988 Acc: 0.4785
val Loss: 1.1644 Acc: 0.4747

Epoch 10/500
train Loss: 0.8656 Acc: 0.6501
val Loss: 1.0207 Acc: 0.5494

Epoch 20/500
train Loss: 0.7954 Acc: 0.6735
val Loss: 1.0128 Acc: 0.6047

Epoch 30/500
train Loss: 0.7630 Acc: 0.6853
val Loss: 0.8073 Acc: 0.6689

Epoch 40/500
train Loss: 0.7384 Acc: 0.6936
val Loss: 0.9204 Acc: 0.6108

Epoch 50/500
train Loss: 0.7168 Acc: 0.7038
val Loss: 0.7774 Acc: 0.6849

Epoch 60/500
train Loss: 0.7003 Acc: 0.7091
val Loss: 1.0650 Acc: 0.6030

Epoch 70/500
train Loss: 0.6853 Acc: 0.7153
val Loss: 0.7611 Acc: 0.6821

Epoch 80/500
train Loss: 0.6700 Acc: 0.7213
val Loss: 0.7987 Acc: 0.6724

Epoch 90/500
train Loss: 0.6545 Acc: 0.7289
val Loss: 0.8290 Acc: 0.6721

Epoch 100/500
train Loss: 0.6431 Acc: 0.7320
val Loss: 0.8548 Acc: 0.6577

Epoch 110/500
train Loss: 0.6283 Acc: 0.7389
val Loss: 0.7877 Acc: 0.6783

Epoch 120/500
train Loss: 0.6187 Acc: 0.7421
val Loss: 0.9982 Acc: 0.6213

Epoch 130/500
train Loss: 0.6060 Ac

In [7]:
# Regression pre-training

model_name = 'LSTM_FCNs_multi'
model_params = config.model_config[model_name]

model_params['parameter']['input_size'] = input_size
model_params['parameter']['alpha'] = 0 # cls = alpha, reg = 1-alpha
model_params['best_model_path'] = f'./ckpt/{SOURCE_DATASET}/lstm_fcn_multi_{seq_len}_pre_reg.pt'

data_source = mt.Multilearning(model_params,'self')
data_source.build_model()

best_model = data_source.train_model(train_x, train_y, valid_x, valid_y)  # 모델 학습

os.makedirs(f'./ckpt/{SOURCE_DATASET}/', exist_ok=True)
data_source.save_model(best_model, best_model_path=model_params["best_model_path"])  # 모델 저장


Epoch 1/500
train Loss: 0.8965 Acc: 0.4179
val Loss: 0.8960 Acc: 0.4377

Epoch 10/500
train Loss: 0.7783 Acc: 0.2221
val Loss: 0.7751 Acc: 0.2074

Epoch 20/500
train Loss: 0.6248 Acc: 0.1843
val Loss: 0.6252 Acc: 0.1682

Epoch 30/500
train Loss: 0.4956 Acc: 0.1812
val Loss: 0.4962 Acc: 0.1751

Epoch 40/500
train Loss: 0.4279 Acc: 0.1816
val Loss: 0.4335 Acc: 0.1780

Epoch 50/500
train Loss: 0.4175 Acc: 0.1853
val Loss: 0.4249 Acc: 0.1739

Epoch 60/500
train Loss: 0.4173 Acc: 0.1490
val Loss: 0.4249 Acc: 0.1617

Epoch 70/500
train Loss: 0.4172 Acc: 0.1479
val Loss: 0.4249 Acc: 0.1385

Epoch 80/500
train Loss: 0.4173 Acc: 0.1426
val Loss: 0.4249 Acc: 0.1373

Epoch 90/500
train Loss: 0.4171 Acc: 0.1415
val Loss: 0.4249 Acc: 0.1390

Epoch 100/500
train Loss: 0.4172 Acc: 0.1325
val Loss: 0.4249 Acc: 0.1336

Epoch 110/500
train Loss: 0.4170 Acc: 0.1450
val Loss: 0.4249 Acc: 0.1277

Epoch 120/500
train Loss: 0.4168 Acc: 0.1473
val Loss: 0.4249 Acc: 0.1234

Epoch 130/500
train Loss: 0.4168 Ac

# Fine-training

In [3]:
# pre Classification fine-training

model_name = 'LSTM_FCNs_multi'
model_params = config.model_config[model_name]

model_params['parameter']['input_size'] = input_size
model_params['parameter']['alpha'] = 0.5 # cls = alpha, reg = 1-alpha
model_params['best_model_path'] = f'./ckpt/{SOURCE_DATASET}/lstm_fcn_multi_{seq_len}_pre_cls.pt' ## pre-train

data_source = mt.Multilearning(model_params, 'self') ## 자기자신 데이터로 학습 하는 option : self
data_source.build_model()

best_model = data_source.train_model(train_x, train_y, valid_x, valid_y, option='target')  # 모델 학습

os.makedirs(f'./ckpt/{SOURCE_DATASET}/', exist_ok=True)
data_source.save_model(best_model, best_model_path=f'./ckpt/{SOURCE_DATASET}/lstm_fcn_pre_cls_fine.pt')  # 모델 저장


Epoch 1/500
train Loss: 0.6975 Acc: 0.7953
val Loss: 0.8778 Acc: 0.6664

Epoch 10/500
train Loss: 0.6354 Acc: 0.7991
val Loss: 1.0414 Acc: 0.5470

Epoch 20/500
train Loss: 0.5764 Acc: 0.8014
val Loss: 0.7580 Acc: 0.6948

Epoch 30/500
train Loss: 0.5260 Acc: 0.8040
val Loss: 0.7173 Acc: 0.6623

Epoch 40/500
train Loss: 0.4891 Acc: 0.8044
val Loss: 0.9273 Acc: 0.5653

Epoch 50/500
train Loss: 0.4634 Acc: 0.8068
val Loss: 0.6815 Acc: 0.6776

Epoch 60/500
train Loss: 0.4518 Acc: 0.8098
val Loss: 0.7367 Acc: 0.6551

Epoch 70/500
train Loss: 0.4472 Acc: 0.8110
val Loss: 0.6893 Acc: 0.6363

Epoch 80/500
train Loss: 0.4395 Acc: 0.8136
val Loss: 1.0120 Acc: 0.5924

Epoch 90/500
train Loss: 0.4378 Acc: 0.8160
val Loss: 0.6410 Acc: 0.6894

Epoch 100/500
train Loss: 0.4368 Acc: 0.8155
val Loss: 0.6824 Acc: 0.6556

Epoch 110/500
train Loss: 0.4322 Acc: 0.8209
val Loss: 0.7379 Acc: 0.6493

Epoch 120/500
train Loss: 0.4300 Acc: 0.8181
val Loss: 0.6162 Acc: 0.7122

Epoch 130/500
train Loss: 0.4295 Ac

In [4]:
# pre Regression fine-training

model_name = 'LSTM_FCNs_multi'
model_params = config.model_config[model_name]

model_params['parameter']['input_size'] = input_size
model_params['parameter']['alpha'] = 0.5 # cls = alpha, reg = 1-alpha
model_params['best_model_path'] = f'./ckpt/{SOURCE_DATASET}/lstm_fcn_multi_{seq_len}_pre_reg.pt' ## pre-train

data_source = mt.Multilearning(model_params, 'self') ## 자기자신 데이터로 학습 하는 option : self
data_source.build_model()

best_model = data_source.train_model(train_x, train_y, valid_x, valid_y, option='target')  # 모델 학습

os.makedirs(f'./ckpt/{SOURCE_DATASET}/', exist_ok=True)
data_source.save_model(best_model, best_model_path=f'./ckpt/{SOURCE_DATASET}/lstm_fcn_pre_reg_fine.pt')  # 모델 저장


Epoch 1/500
train Loss: 1.0570 Acc: 0.4685
val Loss: 1.0424 Acc: 0.4843

Epoch 10/500
train Loss: 0.8391 Acc: 0.6460
val Loss: 0.8411 Acc: 0.6414

Epoch 20/500
train Loss: 0.7500 Acc: 0.6765
val Loss: 0.8024 Acc: 0.6346

Epoch 30/500
train Loss: 0.6837 Acc: 0.6870
val Loss: 0.8025 Acc: 0.5846

Epoch 40/500
train Loss: 0.6281 Acc: 0.6940
val Loss: 0.8123 Acc: 0.6017

Epoch 50/500
train Loss: 0.5855 Acc: 0.7049
val Loss: 0.8683 Acc: 0.5565

Epoch 60/500
train Loss: 0.5629 Acc: 0.7085
val Loss: 0.6599 Acc: 0.6616

Epoch 70/500
train Loss: 0.5507 Acc: 0.7170
val Loss: 0.6158 Acc: 0.6670

Epoch 80/500
train Loss: 0.5403 Acc: 0.7262
val Loss: 0.6718 Acc: 0.6231

Epoch 90/500
train Loss: 0.5342 Acc: 0.7301
val Loss: 0.6967 Acc: 0.5996

Epoch 100/500
train Loss: 0.5271 Acc: 0.7377
val Loss: 0.7648 Acc: 0.5997

Epoch 110/500
train Loss: 0.5212 Acc: 0.7447
val Loss: 0.7136 Acc: 0.6361

Epoch 120/500
train Loss: 0.5147 Acc: 0.7468
val Loss: 0.6969 Acc: 0.6315

Epoch 130/500
train Loss: 0.5105 Ac